In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings; warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('Data/Preprocessed.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0.813399,148.0,72.0,0.696814,0.183531,0.260117,0.821764,1.364180,1
1,-0.833906,85.0,66.0,0.036596,0.183531,-0.843284,-0.168409,0.126452,0
2,1.188996,183.0,64.0,0.054048,0.183531,-1.457745,0.935284,0.230161,1
3,-0.833906,89.0,66.0,-0.674132,-0.791683,-0.586404,-1.298725,-1.480075,0
4,-1.603317,137.0,40.0,0.696814,0.343910,1.462682,2.336680,0.327328,1


In [7]:
features = df.drop('Outcome', axis=1)
label = df['Outcome']
print(features.shape, label.shape)

(768, 8) (768,)


In [5]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=21)

In [14]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression()

accuracy = []
print('[INFO] Accuracy with LogisticRegression...')
for train_index, test_index in skf.split(features, label):
    # data split
    x_train_fold, x_test_fold = features.iloc[train_index], features.iloc[test_index]
    y_train_fold, y_test_fold = label.iloc[train_index], label.iloc[test_index]
    
    # model training and its accuracy
    lr_model.fit(x_train_fold, y_train_fold)
    accuracy.append(lr_model.score(x_test_fold, y_test_fold))

print(accuracy)

[INFO] Accuracy with LogisticRegression...
[0.7662337662337663, 0.7467532467532467, 0.7597402597402597, 0.7254901960784313, 0.8169934640522876]


In [28]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, min_samples_leaf=2, n_jobs=-1, random_state=21)

accuracy = []
print('[INFO] Accuracy with Random Forest...')
for train_index, test_index in skf.split(features, label):
    # data split
    x_train_fold, x_test_fold = features.iloc[train_index], features.iloc[test_index]
    y_train_fold, y_test_fold = label.iloc[train_index], label.iloc[test_index]
    
    # model training and its accuracy
    rf_model.fit(x_train_fold, y_train_fold)
    accuracy.append(rf_model.score(x_test_fold, y_test_fold))

print(accuracy)

[INFO] Accuracy with Random Forest...
[0.7467532467532467, 0.7402597402597403, 0.7467532467532467, 0.7516339869281046, 0.8104575163398693]


In [37]:
from sklearn.svm import SVC
svm_model = SVC(kernel='rbf', degree=5, random_state=0)

accuracy = []
print('[INFO] Accuracy with SVM...')
for train_index, test_index in skf.split(features, label):
    # data split
    x_train_fold, x_test_fold = features.iloc[train_index], features.iloc[test_index]
    y_train_fold, y_test_fold = label.iloc[train_index], label.iloc[test_index]
    
    # model training and its accuracy
    svm_model.fit(x_train_fold, y_train_fold)
    accuracy.append(svm_model.score(x_test_fold, y_test_fold))

print(accuracy)

[INFO] Accuracy with SVM...
[0.7337662337662337, 0.7532467532467533, 0.7532467532467533, 0.7320261437908496, 0.7254901960784313]
